In [1]:
# import
import pandas as pd
import numpy as np
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew, kurtosis
import utils

import NN
import os
from torch import load as load_model
import config as cfg

In [2]:
LSTM_RESULTS_PATH = "models_norm/LSTM_preds"
MLP_RESULTS_PATH = "models_norm/MLP_preds"
RNN_RESULTS_PATH = "models_norm/RNN_preds"
PSN_RESULTS_PATH = "models_norm/PSN_preds"

In [3]:
# all data
dia_df = pd.read_csv("data/DIA.csv")
qqq_df = pd.read_csv("data/QQQ.csv")
spy_df = pd.read_csv("data/SPY.csv")

# risk free
riskfree = pd.read_csv("data/Tbill/Valid.csv")
riskfreeDf = riskfree[['Date', 'Adj Close']].copy()
riskfreeDf['Date'] = pd.to_datetime(riskfreeDf['Date'])
riskfreeDf.set_index('Date', inplace=True)
riskfreeDf.rename(columns={'Adj Close':'Risk_free'}, inplace=True)
riskfreeRate = riskfreeDf["Risk_free"].mean()

# adding suffix to all df
dia_df.columns=dia_df.columns.map(lambda x : x+'_dia' if x !='Date' else x)
qqq_df.columns=qqq_df.columns.map(lambda x : x+'_qqq' if x !='Date' else x)
spy_df.columns=spy_df.columns.map(lambda x : x+'_spy' if x !='Date' else x)

dfs = [dia_df, qqq_df, spy_df]

df = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)

# set date column as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# keeping only the correct date 03/01/2011 to 13/04/2015
Total_df = df.loc[('2011-01-03' <= df.index) & (df.index <= '2015-04-13')]
Training_df = df.loc[('2011-01-03' <= df.index) & (df.index <= '2012-12-31')]
Test_df = df.loc[('2013-01-02' <= df.index) & (df.index <= '2013-12-31')]
Out_of_sample_df = df.loc[('2014-01-02' <= df.index) & (df.index <= '2015-04-13')]

# ptf portfolio
Total_adj_close_df = Total_df[['Adj Close_dia', 'Adj Close_qqq', 'Adj Close_spy']].copy()
Total_adj_close_df[['prev_adj_Close_dia', 'prev_adj_Close_qqq', 'prev_adj_Close_spy']] = Total_adj_close_df[['Adj Close_dia', 'Adj Close_qqq', 'Adj Close_spy']].shift(periods=1)

# weeklyDf = Total_adj_close_df.resample('D').interpolate()[::7]
weeklyDf = Total_adj_close_df.copy()

weeklyDf['DIA'] = np.log(weeklyDf['Adj Close_dia']) - np.log(weeklyDf['prev_adj_Close_dia'])
weeklyDf['QQQ'] = np.log(weeklyDf['Adj Close_qqq']) - np.log(weeklyDf['prev_adj_Close_qqq'])
weeklyDf['SPY'] = np.log(weeklyDf['Adj Close_spy']) - np.log(weeklyDf['prev_adj_Close_spy'])

# keeping only Out-of-Sample
weeklyDf = weeklyDf[(weeklyDf.index >= '2014-01-01')]
weeklyDf = weeklyDf[['DIA', 'QQQ', 'SPY']]
weeklyDf['1/N'] = (1/3) * (weeklyDf['DIA'] + weeklyDf['QQQ'] + weeklyDf['SPY'])

In [4]:
weeklyDf

,DIA,QQQ,SPY,1/N
Date,,,,
2014-01-02,-0.008253,-0.007876,-0.009630,-0.008586
2014-01-03,0.001705,-0.007245,-0.000164,-0.001902
2014-01-06,-0.003046,-0.003700,-0.002902,-0.003216
2014-01-07,0.006568,0.009225,0.006123,0.007305
2014-01-08,-0.003765,0.002179,0.000218,-0.000456
...,...,...,...,...
2015-04-07,-0.000952,-0.001700,-0.002650,-0.001767
2015-04-08,0.001848,0.007908,0.003371,0.004376
2015-04-09,0.003629,0.006263,0.004414,0.004769


# Trading perf

In [14]:
Table7 = pd.DataFrame({})

for ptf in ['DIA', 'SPY', 'QQQ', '1/N']:    
    weeklyDf[ ptf + '_Cumulative_return' ] = (1 + weeklyDf[ ptf ]).cumprod()
    
    annualizedReturn = weeklyDf[ ptf ].cumsum().to_list()[-1] * 252 * (1/len(weeklyDf))
    annualizedVol = weeklyDf[ ptf ].std() * np.sqrt(252)
    downsideVol = weeklyDf.loc[weeklyDf[ ptf ] < 0][ ptf ].std() * np.sqrt(252)
    maximumDrawdown = 100 * utils.MDD(weeklyDf, ptf + "_Cumulative_return", len(weeklyDf))
    sharpeRatio = (100 * annualizedReturn - riskfreeRate)/ (100 * annualizedVol)
    sortinoRatio = (100 * annualizedReturn - riskfreeRate)/ (100 * downsideVol)

    Table = pd.DataFrame({'Strategies':[ptf],
                          'Realized return (%)':[100 * annualizedReturn],
                          'Sharpe Ratio':[sharpeRatio],
                          'Sortino Ratio':[sortinoRatio],
                          'Max drawdown (%)':[maximumDrawdown]
                         })    
    Table7 = pd.concat([Table7,Table],ignore_index=True)
Table7.set_index()

In [15]:
Table7

,Strategies,Realized return (%),Sharpe Ratio,Sortino Ratio,Max drawdown (%)
0,DIA,8.543654,0.531833,0.739671,-6.931405
1,SPY,11.610182,0.785750,1.087035,-7.375937
2,QQQ,17.108159,1.053613,1.420047,-8.384762
3,1/N,12.420665,0.833813,1.142637,-7.456279


In [8]:
ptf

'1/N'